In [1]:
from bayespy.nodes import Categorical, Mixture, MultiMixture
from bayespy.inference import VB
%matplotlib inline

In [10]:
# 2. Water sprinkler example using BayesPy
exp = 1e-20 # BayesPy has problems with 1.0 -> substract a small value
cloudy = Categorical([0.5, 0.5])
rain = Mixture(
    cloudy,
    Categorical, 
    [
        [0.8, 0.2], # p(rain | not cloudy) = 0.2
        [0.2, 0.8], # p(rain | cloudy) = 0.8
    ]
)
sprinkler = Mixture(
    cloudy,
    Categorical, 
    [
        [0.5, 0.5], # p(sprinkler on | not cloudy) = 0.5
        [0.9, 0.1], # p(sprinkler on | cloudy) = 0.1
    ]
)
wetGrass = MultiMixture(
    [rain, sprinkler],
    Categorical,
    [
        # No rain
        [
            # No sprinkler
            [1.0 - exp, 0.0 + exp],
            # Yes sprinkler
            [0.1, 0.9]
        ],
        # Yes rain
        [
            # No sprinkler
            [0.1, 0.9],
            # Yes sprinkler
            [0.01, 0.99]
        ]
    ]
)

In [11]:
wetGrass.observe(1)

Q = VB(wetGrass, rain, sprinkler, cloudy)
Q.update(repeat=100, verbose=True)

print('P(s=1|w=1) =', sprinkler.get_moments()[0][1])
print('P(r=1|w=1) =', rain.get_moments()[0][1])

Iteration 1: loglike=-8.859004e-01 (0.015 seconds)
Iteration 2: loglike=-8.434570e-01 (0.012 seconds)
Iteration 3: loglike=-8.430205e-01 (0.022 seconds)
Iteration 4: loglike=-8.430174e-01 (0.024 seconds)
Converged at iteration 4.
P(s=1|w=1) = 0.148459431093
P(r=1|w=1) = 1.0
